In [ ]:
# 뉴스기사 크롤링
# 제목 링크 날짜 퍼블리션
# 동적 크롤링, 셀리니움(느리고 방대한데이터에 부적합, 봇으로판단돼서 차단당할수있음_)
# 주의 해야할점
# 1. 너무 많은 요청 보내지않기 
# 2. 사람인것처럼 요청 보내기 User-Agent

In [ ]:
import requests
from bs4 import BeautifulSoup
# 2008.01.02&de=2008.01.02 여기부분이 날짜 변경 부분
# 그러면 여기에 날짜를 정해줄수있는 방법이있어야하나?
# 일단 넣어보자
find = "기준금리"
start_date = "2008.01.02"
end_date = "2008.01.02"
page =1 
url = f"https://search.naver.com/search.naver?where=news&query={find}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{start_date.replace('.', '')}to{end_date.replace('.', '')}&is_sug_officeid=0&office_category=3&service_area=0&start={page}"

print(url)
res = requests.get(url, headers = {'User-Agent' : "mozilla 5.0"})
res.status_code, res.text.find('큰기쁨예금')

https://search.naver.com/search.naver?where=news&query=기준금리&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2008.01.02&de=2008.01.02&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20080102to20080102&is_sug_officeid=0&office_category=3&service_area=0&start=1


(200, 220805)

In [ ]:
import requests
from bs4 import BeautifulSoup

find = "기준금리"
start_date = "2008.01.02"
end_date = "2008.01.02"
page = 1  
articles = []

while True:
    url = f"https://search.naver.com/search.naver?where=news&query={find}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{start_date.replace('.', '')}to{end_date.replace('.', '')}&is_sug_officeid=0&office_category=3&service_area=0&start={page}"
    # f스트링으로 써야 저장한 변수가 적용됨
    # User-Agent 추가 (네이버에서 봇으로 차단되지 않도록)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
    
    # 페이지 요청
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    
    # 기사 목록 가져오기
    news_list = soup.select(".news_area")  
    # .bx 너무 많은 부분이 떠버림
    
    # 더 이상 기사가 없으면 종료
    if not news_list:
        break

    for news in news_list:
        title_tag = news.select_one(".news_tit")  # 제목 태그
        press_tag = news.select_one(".info.press")  # 신문사 태그
        date_tag = news.select_one("span.info")  # 날짜 태그 
        
        if title_tag and press_tag and date_tag:
            title = title_tag.text.strip()  # 기사 제목
            link = title_tag["href"]  # 기사 링크
            news_agency = press_tag.text.strip()  # 신문사 이름
            article_date = date_tag.text.strip()  # 날짜
            
            articles.append((title, news_agency, article_date, link))
    
    print(f"{page}번째 페이지 크롤링 완료, 기사 개수: {len(news_list)}")
    
    # 다음 페이지로 이동
    page += 10  # 10개씩 넘어감 
    # 5개로 줄여봤는데 중복으로 기사를 가져옴

    import time
    import random
    # 차단 방지 1~2초 쉬어 주기
    sleep_time = random.uniform(1,2) # 몰라서 물어봄...
    time.sleep(sleep_time)

for idx,(title, news_agency, article_date, link) in enumerate(articles, 1):
    print(f"'{article_date}','{news_agency}','{title}', '{link}'") 
    # csv 로 저장하려고 , 로 넣었는데 이거 기사제목에도 , 이게 붙지않나?
    # 그러면 각각 문자열로 바꿔주면 기사 제목에 , 도 csv 에선 문자열로 인식
    # 이제 이걸 csv로 저장
    


1번째 페이지 크롤링 완료, 기사 개수: 10
11번째 페이지 크롤링 완료, 기사 개수: 10
21번째 페이지 크롤링 완료, 기사 개수: 4
'2008.01.02.','파이낸셜뉴스','외환銀 ‘큰기쁨예금’ 우대금리 제공', 'http://www.fnnews.com/view?ra=Sent0701m_01A&corp=fnnews&arcid=080101220028&cDateYear=2008&cDateMonth=01&cDateDay=01&'
'2008.01.02.','한국경제','금리 연초부터 급등', 'http://www.hankyung.com/news/app/newsview.php?aid=2008010219021'
'2008.01.02.','이데일리','CD금리 상승세 지속..5.8%대 중반도 임박', 'http://www.edaily.co.kr/news/newspath.asp?newsid=02210726586273784'
'2008.01.02.','매일경제','금리 껑충 … 국고채 0.12%P올라', 'http://news.mk.co.kr/news_forward.php?no=3138&year=2008'
'2008.01.02.','한국경제','정기예금 금리 두달새 1%P 상승‥ 3개월만기 최고 5.9%', 'http://www.hankyung.com/news/app/newsview.php?aid=2008010211601'
'2008.01.02.','이데일리','씨티은행 주택담보대출 금리 인상', 'http://www.edaily.co.kr/news/newspath.asp?newsid=01682646586273784'
'2008.01.02.','이데일리','CD금리 상승 지속..6년7개월 최고행진', 'http://www.edaily.co.kr/news/newspath.asp?newsid=01804006586273784'
'2008.01.02.','아시아경제','亞 증시, 올해도 '자신만만'', 'http://www.asiae.co.kr/uhtml/read.jsp?i